# ANITA astroinformatics summer school 2019 - "Rise of the machines"

## Part I - data cleaning and exploration

This lesson is adapted from the [Data Carpentry Ecology lesson](http://www.datacarpentry.org/python-ecology-lesson/)

We'll be using a gitter chat room to ask questions and chat:
https://gitter.im/ANITA-astroinformatics-school/2019-school

After finishing the course an updated notebook with the answers will be made available in the github repo.

## Table of Contents

1. [Pandas and DataFrames](#Pandas-and-DataFrames)
2. [Dealing With Data In Pandas](#Dealing-With-Data-In-Pandas)
3. [Calculating statistics in a pandas DataFrame](#Calculating-statistics-in-a-pandas-DataFrame)
6. [Indexing & Slicing in Pandas](#Indexing-&-Slicing-in-Pandas)
5. [Cleaning our data for machine learning](#Cleaning-our-data-for-machine-learning)
4. [Exploratory analysis](#Exploratory-analysis)
7. [Supplement: Merging DataFrames](#Supplement:-Merging-DataFrames)

### Required libraries

This notebook uses several Python packages that come standard with the [Anaconda Python distribution](http://continuum.io/downloads). The primary libraries that we'll be using are:

* **pandas**: a DataFrame structure to store data in memory and work with it easily and efficiently.
* **seaborn**: a advanced statistical plotting library.

To make sure you have all of the packages you need, install them with `conda`:

    conda install pandas seaborn

`conda` may ask you to update some of the packages if you don't have the most recent version. Allow it to do so.

Alternatively, if you can install the packages with [pip](https://pip.pypa.io/en/stable/installing/) (a Python package manager):

    pip install pandas seaborn

# Pandas and DataFrames


One of the best options for working with tabular data in Python is to use the
[Python Data Analysis Library](http://pandas.pydata.org/) (a.k.a. Pandas). The
Pandas library provides data structures, produces high quality plots with
[matplotlib](http://matplotlib.org/) and integrates nicely with other libraries
that use [NumPy](http://www.numpy.org/) (which is another Python library) arrays.

Each time we call a function that's in a library, we use the syntax
`LibraryName.FunctionName`. Adding the library name with a `.` before the
function name tells Python where to find the function. 

We will import Pandas as `pd`, a common abbreviation of this library.

## So What's a DataFrame?

A DataFrame is a 2-dimensional (labeled) data structure that stores data of different
types (including characters, integers, floating point values, factors and more) in columns. 

It is similar to a spreadsheet or an SQL table or the `data.frame` in R. 

A DataFrame always has an index (0-based) and works best with *tidy data*.
According to [Hadley Wickham](http://vita.had.co.nz/papers/tidy-data.html):
> Tidy datasets are easy to manipulate, model and visualize, and have a specific structure: each variable is a column, each observation is a row, and each type of observational unit is a table. 

More info on the pandas DataFrame: https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe

## Reading data into a pandas DataFrame

### Our Data 

For this lesson, we will be using [Galaxy Zoo DR1 data](https://www.google.com/search?q=galaxy+zoo&ie=utf-8&oe=utf-8&client=firefox-b-ab). Galaxy Zoo is described in Lintott et al. 2008, MNRAS, 389, 1179 and the data release is described in Lintott et al. 2011, 410, 166.

The table we use is an adapted version of Table 2, listing classifications of galaxies which have spectra included in SDSS Data Release 7. The debiased fraction of the votes in elliptical and spiral categories is given, along with flags identifying systems as classified as spiral, elliptical or uncertain.


| Column           | Description                             |
|------------------|-----------------------------------------|
| id               | SDSS ID, objects taken from DR7         |
| ra               | Right Ascension  (HMS)                  |
| dec              | Declination (DMS)                       |
| nvote            | number of votes                         |
| p_e              | debiased vote fraction Ellipticals      |
| p_s              | debiased vote fraction all Spirals      |
| type             | whether final vote is E, S or U         |
| class            | spiral or elliptical class, eg E0 or CW |

Galaxies flagged as ‘elliptical’ or ‘spiral’ require 80 per cent of the vote in that category after the debiasing procedure has been applied; all other galaxies are flagged ‘uncertain’.
Note, the elliptical class is randomly assigned. The spiral class is based on the highest vote fraction of the spiral classes in the Galaxy Zoo DR 1 Table 2 data.

### Starting in the same spot

To help the lesson run smoothly, let's ensure everyone is in the same directory.
This should help us avoid path and file name issues. 

At this time please navigate to the workshop directory (ie, the directory with the notebooks in them). 
If you working in IPython Notebook be sure that you start your notebook in the workshop directory.


A quick aside, there are Python libraries like [OS
Library](https://docs.python.org/3/library/os.html) that can work with our
directory structure, however, that is not our focus today.


If you **need to change your directory** ```import os``` and use ```os.chdir```

Or you can use **%** to access the command line, e.g. ```% cd folder_name```

--- 

We will begin by locating and reading our Galaxy Zoo data which is in a CSV file.
We can use Pandas' `read_csv` function to pull the file directly into a
[DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe).

pandas has easy to use functions to read in data from various data sources, including CSV and HDF5, see [here](http://pandas.pydata.org/pandas-docs/version/0.24.0rc1/io.html#) for more info.


In [ ]:
#if you need to change your directory
import os
print(os.getcwd())

#os.chdir("notebooks/") #make sure you enter the correct fille path

In [ ]:
os.getcwd()

In [ ]:
import pandas as pd
#check your version, we need v0.19 or higher
pd.__version__

**Note** you will need pandas version 0.19 or higher. Please update the package if required!

In [ ]:
# note that pd.read_csv is used because we imported pandas as pd
gal_df=pd.read_csv('data/GalaxyZoo1.csv')

Let's check the data type of the data stored in `gal_df` using the `type` method. 

Both the `type` method and `__class__` attribute tell us that `gal_df` is `<class 'pandas.core.frame.DataFrame'>`.

In [ ]:
type(gal_df)
# or gal_df.__class__

To check the data types for each column in our DataFrame, we can use `gal_df.dtypes`.

In [ ]:
gal_df.dtypes

Pandas and base Python use slightly different names for data types. More on this
is in the table below:

| Pandas Type | Native Python Type | Description |
|-------------|--------------------|-------------|
| object | string | The most general dtype. Will be assigned to your column if column has mixed types (numbers and strings). |
| int64  | int | Numeric characters. 64 refers to the memory allocated to hold this character. |
| float64 | float | Numeric characters with decimals. If a column contains numbers and NaNs(see below), pandas will default to float64, in case your missing value has a decimal. |
| datetime64, timedelta[ns] | N/A (but see the [datetime] module in Python's standard library) | Values meant to hold time data. Look into these for time series experiments. |

[datetime]: http://doc.python.org/2/library/datetime.html

**There are multiple other methods that can be used to summarize and access the data stored in DataFrames**. 

Let's try out a few:

### Your turn

Try out the methods below to see what they return.

1. `gal_df.columns`.
2. `gal_df.head()`. Also, what does `gal_df.head(15)` do?
3. `gal_df.tail()`.
4. `gal_df.shape`. Take note of the output of the shape method. What format does it return the shape of the DataFrame in?

HINT: [More on tuples, here](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences).

We can see that there were 667944 rows and 8 columns parsed.
It looks like  the `read_csv` function in Pandas  read our file properly.

Now we can start manipulating our data. 


# Dealing With Data In Pandas

[[ go back to the top ]](#Table-of-Contents)

## Selecting Data Using Labels (Column Headings)

We use square brackets `[]` to select a subset of a Python object. For example,
we can select all of the data from a column named `type` from the `gal_df`
DataFrame by name:

```python
gal_df['type']
# this syntax, calling the column as an attribute, gives you the same output
gal_df.type
```

We can also create an new object that contains the data within the `type`
column as follows:

```python
# create an object named gal_types that only contains the `types` column
gal_types = gal_df['types']
```

We can pass a list of column names too, as an index to select columns in that
order. This is useful when we need to reorganize our data.

**NOTE:** If a column name is not contained in the DataFrame, an exception
(error) will be raised.

```python
# select the type and class columns from the DataFrame
gal_df[['type', 'class']]
# what happens when you flip the order?
gal_df[['class', 'type']]
#what happens if you ask for a column that doesn't exist?
gal_df['types']
```


## Changing column types and content


To modify the format of values within our data frame we can use the ```astype``` function (also remember in pandas the type is `float64`).

Don't forget this is a function within pandas so we use it with a `.`, for example to convert 
the `nvote` field to floating point values we would run:

In [ ]:
# convert the nvote field from an integer to a float
gal_df['nvote']=gal_df['nvote'].astype('float64')

In [ ]:
gal_df.dtypes

What happens if we try to convert probability values to integers?

In [ ]:
gal_df['p_e']=gal_df['p_e'].astype('int64')

Notice that this throws a value error: `ValueError: Cannot convert non-finite values to integer`. 

If we look at the `p_e` column in our data we notice that
there are NaN (**N**ot **a** **N**umber) values. *NaN* values are undefined
values that cannot be represented mathematically. Pandas, for example, will read
an empty cell in a CSV or Excel sheet as a NaN. NaNs have some desirable
properties: if we were to average the `weight` column without replacing our NaNs,
Python would know to skip over those cells.

_Note: older pandas version do not know how to handle NaN, please update to v0.19_

Check your pandas version using `pd.__version__`, if you need to update open a bash shell
and type ```conda update pandas```, or if using pip ```pip install -U pandas```.



----

Pandas generally vectorises operations, which means that looping through your data is not necessary.
For example, we could perform a mathematical operation on an entire column of our data.

In [ ]:
print(gal_df.nvote.head())
# multiply all votes by 2


This is equivalent to:

---

## Missing Data Values - NaN

Dealing with missing data values is always a challenge. It's sometimes hard to
know why values are missing - was it because of a data entry error? Or data that
someone was unable to collect? Should the value be 0? We need to know how
missing values are represented in the dataset in order to make good decisions.
If we're lucky, we have some metadata that will tell us more about how null
values were handled.

For instance, in some disciplines, like astronomy or remote Sensing, missing data values are
often defined as -9999. Having a bunch of -9999 values in your data could really
alter numeric calculations. Often in spreadsheets, cells are left empty where no
data are available. Pandas will, by default, replace those missing values with
NaN. However, **it is good practice to get in the habit of intentionally marking
cells that have no data, with a no data value!** That way there are no questions
in the future when you (or someone else) explores your data.

### Where Are the NaN's?

Let's explore the NaN values in our data a bit further. 
First, let's figure out how many rows contain NaN values for the elliptical probabilities. 
We can do this by identifying how many rows have a NULL value (`.isnull`) or by counting the number of rows that have a meaningful value (e.g., p_e>0):

This creates a mask we can use to select the NULL values only. We'll talk more about [subsetting](#Indexing-&-Slicing-in-Pandas) data using masks later.


In [ ]:
# Number of missing values


We can replace all NaN values with zeroes using the `.fillna()` method (after
making a copy of the data so we don't lose our work).

However, NaN and 0 yield different analysis results. The mean value when NaN
values are replaced with 0 is different from when NaN values are simply thrown
out or ignored.

In [ ]:
#replace nan with 0


In [ ]:
#check mean, how does it differ from before?


We can fill NaN values with any value that we choose. The code below fills all
NaN values with a mean of all probability values.

```python
 df1['p_e'] = gal_df['p_e'].fillna(gal_df['p_e'].mean())
```

**Note**: when we use the `.mean()` function, pandas automatically ignores the NaN.


We could also chose to create a subset of our data, only keeping rows that do
not contain NaN values, using `.dropna()` method.

In [ ]:
# drop NaN and compare mean before and after


**The point is to make conscious decisions about how to manage missing data.** 
This is where we think about how our data will be used and how these values will
impact the scientific conclusions made from the data.

Python gives us all of the tools that we need to account for these issues. We
just need to be cautious about how the decisions that we make impact scientific
results.


**For more info on how to handle missing data in pandas, check out the [docs](http://pandas.pydata.org/pandas-docs/version/0.24.0rc1/missing_data.html)**

# Calculating statistics in a Pandas DataFrame

[[ go back to the top ]](#Table-of-Contents)

We've read our data into Python. Next, let's perform some quick summary
statistics to learn more about the data that we're working with. We might want
to know how many galaxies of a specific type we have, or what the average number of votes was. 
We can perform summary stats quickly using groups. But first we need to figure out what we want to group by.

Let's begin by exploring our data:

In [ ]:
#column names


Let's get a list of all the galaxy types. The `pd.unique` function tells us all of
the unique values in the `type` column:

Another useful function is `.nunique()` which returns the number of unique entries in the specifed column(s)

## Summary Statistics
The Pandas function `.describe()` will return descriptive stats including: mean,
median, max, min, std and count for a particular column in the data. Pandas'
`describe` function will only return summary values for columns containing
numeric data.
We can calculate basic statistics for all numeric records in a single column or the entire dataframe using the
syntax below:

We can also extract one specific metric if we wish:

```python
gal_df['nvote'].min()
gal_df['nvote'].max()
gal_df['nvote'].mean()
gal_df['nvote'].std()
gal_df['nvote'].count()
```

## Groups in pandas and summary stats

But if we want to summarize by one or more variables, for example galaxy type, we can
use Pandas' `.groupby` method. Once we've created a groupby DataFrame, we
can quickly calculate summary statistics by a group of our choice.

In [ ]:
# summary statistics for all numeric columns by type
grouped_data = gal_df.groupby('type')
grouped_data.describe()

In [ ]:
# provide the mean for each numeric column by type


The `groupby` command is powerful in that it allows us to quickly generate
summary stats.


Let's next count the number of galaxies for each type. We can do this in a few
ways, but we'll use `groupby` combined with a `count()` method.


```python
# count the number of samples by type
type_counts = gal_df.groupby('type')['id'].count()
```

Or, we can also count just the rows that have type='U':

```python
gal_df.groupby('type')['id'].count()['U']
```

In [ ]:
gal_df.groupby('type')['id'].count()

In [ ]:
gal_df.groupby('type')['id'].count()['U']

### Your turn

1. What happens when you group by two columns using the following syntax and
    then grab mean values:
	- `sorted_data = gal_df.groupby(['type','class'])`
	- `sorted_data.mean()`
2. Get summary statistics on the number of votes for each galaxy class in your data. HINT: you can use the
   following syntax to only create summary statistics for one column in your data
   `name_of_dataframe['name_of_column'].describe()`

Another quick way to get the count of each unique value in a column is the `value_counts()` method:

In [ ]:
# number of entries for each class of galaxy
gal_df['class'].value_counts()

## Quick & Easy Plotting Data Using Pandas

We can plot our summary stats using Pandas, too. Check the [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) for more details.

In [ ]:
# make sure figures appear inline in Jupyter Notebook
%matplotlib inline

# create a scatter plot


In [ ]:
#creating a histogram with .hist plotting option


In [ ]:
# histogram from a groupby operation


In [ ]:
counts_per_class = gal_df.groupby('class')['id'].nunique()

# bar plots



### Your turn

You can choose what kind of plot to create (scatter, boxplot, histogram...).
Remeber to check the [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) for more details.

1. Create a plot of average number votes across all classes of galaxies.
2. Create a plot showing the distribution of number of votes for each type of galaxy
3. Create a plot of number of votes vs p_el for each type of galaxy

In [ ]:
# 1. bar plot of average number of votes for each class


In [ ]:
# 2. box plot of number of votes for each type


In [ ]:
# 3. Scatter plot of number of votes vs p_e grouped by type


# Indexing & Slicing in Pandas

[go back to the top](#Table-of-Contents)

We often want to work with subsets of a **DataFrame** object. There are
different ways to accomplish this including: using labels (ie, column headings - as used previously),
numeric ranges or specific x,y index locations.


**REMINDER**: Python Uses 0-based Indexing


![indexing diagram](https://datacarpentry.org/python-ecology-lesson/fig/slicing-indexing.png)

![slicing diagram](https://datacarpentry.org/python-ecology-lesson/fig/slicing-slicing.png)


## Slicing using the `[]` operator

Slicing using the `[]` operator selects a set of rows and/or columns from a
DataFrame. To slice out a set of rows, you use the following syntax:
`data[start:stop]`. When slicing in pandas the start bound is included in the
output. The stop bound is one step BEYOND the row you want to select. So if you
want to select rows 0, 1 and 2 your code would look like this:

```python
# select rows 0,1,2 (but not 3)
gal_df[0:3]

# select the first 5 rows (rows 0,1,
2,3,4)
gal_df[:5]

# select the last element in the list
gal_df[-1:]
```

To access columns we will need to call them by their labels, if you want more than one column you need to supply a list of column names.

```python
# selecting the if column
gal_df['id']

# selecting id and type
gal_df[['id','type']]
```



In [ ]:
gal_df[0:5:2]

In [ ]:
gal_df.columns

In [ ]:
# for a single column we can select a subset of rows by either giving a list of rows or a slice
gal_df['nvote'][[0,1,2]]

In [ ]:
gal_df['nvote'][0:3]

In [ ]:
# for a subset of columns we need to specify a slice of rows
gal_df[['nvote','type']][0:3]

We can also reassign values within subsets of our DataFrame. But before we do that, let's make a 
copy of our DataFrame so as not to modify our original imported data. 

```python
# copy the surveys dataframe so we don't modify the original DataFrame
gal_copy = gal_df

# set the first three rows of data in the DataFrame to 0
gal_copy[0:3] = 0
```

Next, try the following code: 

```python
gal_copy.head()
gal_df.head()
```
What is the difference between the two data frames?

In [ ]:
gal_copy = gal_df

In [ ]:
gal_df.head()

In [ ]:
gal_copy[0:3]=0

In [ ]:
gal_copy.head()

In [ ]:
gal_df.head()

### Referencing Objects vs Copying Objects in Python

We might have thought that we were creating a fresh copy of the `gal_df` objects when we 
used the code `surveys_copy = gal_df`. However the statement  y = x doesn’t create a copy of our DataFrame. 
It creates a new variable y that refers to the **same** object x refers to. This means that there is only one object 
(the DataFrame), and both x and y refer to it. So when we assign the first 3 columns the value of 0 using the 
`surveys_copy` DataFrame, the `gal_df` DataFrame is modified too. To create a fresh copy of the `gal_df`
DataFrame we use the syntax y=x.copy(). But before we have to read the gal_df again because the current version contains the unintentional changes made to the first 3 columns.

```python
gal_df = pd.read_csv("data/GalaxyZoo1.csv")
gal_copy= gal_df.copy()

```

In [ ]:
#read data back in and check it's correct
gal_df = pd.read_csv("data/GalaxyZoo1.csv")
gal_df.head()

In [ ]:
#copy data frame and check the copy
gal_copy = gal_df.copy()
gal_copy.head()

In [ ]:
#modify copy and check both copy and original to see changes
gal_copy[0:3]=0
gal_copy.head()

In [ ]:
gal_df.head()

## Slicing and subsetting: label vs integer-based indexing

We can select specific loctions or ranges of our data in both the row and column directions
using either label or integer-based indexing.

- `loc`: indexing via *labels* (which can be numbers)
- `iloc`: indexing via *integers*


![loc_iloc_subsetting](http://104.236.88.249/wp-content/uploads/2016/10/Pandas-selections-and-indexing.png)


![dataframe_indexing](https://vrzkj25a871bpq7t1ugcgmn9-wpengine.netdna-ssl.com/wp-content/uploads/2019/01/pandas-dataframe-has-indexes.png)

To select an index subset of rows AND columns from our DataFrame, we can use the `iloc`
method. For example, we can select RA, Dec and number of votes (columns 2, 3 and 4 if we
start counting at 1), by slicing our index like this:

```python
gal_df.iloc[0:3, 1:4]
```

**Note**: the order of selection is ROW followed by COLUMN

In [ ]:
gal_df.iloc[0:3, 1:4]

Notice that we asked for a slice from 0:3. This yielded 3 rows of data. When you
ask for 0:3, you are actually telling python to start at index 0 and select rows
0, 1, 2 **up to but not including 3**.

---

Next let's explore subsetting our data using labels.
**Note** When "slicing" labels, the start bound and the stop bound are **included**. 

In [ ]:
# select all columns for rows of index values 0 and 10


**Caution**  When using `loc`, integers *can* also be labels, but they refer to the **index label** and not the position. Thus when you use `loc`, and select rows 0:3, you will get a different result than using
`iloc` to select rows 0:3.

In [ ]:
# using .loc select rows 0:3 and columns ['ra', 'dec', 'nvote']


In [ ]:
# What happens when try to access a row label that does not exist using .loc?


In [ ]:
# Try using iloc instead of loc


NOTE: If using `iloc` labels must be found in the DataFrame or you will get a `KeyError`. Using `loc` (at least for now) you'll get NaN entries returned so be careful!


## Subsetting using masks

A mask can be useful to locate where a particular subset of values exist or
don't exist - for example,  NaN, or "Not a Number" values. To understand masks,
we also need to understand `BOOLEAN` objects in python.

Boolean values include `true` or `false`. So for example

```python
# set x to 5
x = 5
# what does the code below return?
x > 5
# how about this?
x == 5
```
When we ask python what the value of `x > 5` is, we get `False`. This is because x
is not greater than 5 it is equal to 5. To create a boolean mask, you first create the
True / False criteria (e.g. values > 5 = True). Python will then assess each
value in the object to determine whether the value meets the criteria (True) or
not (False). Python creates an output object that is the same shape as
the original object, but with a True or False value for each index location.

You can use the syntax below when querying data from a DataFrame. Experiment
with selecting various subsets of our data.

* Equals: `==`
* Not equals: `!=`
* Greater than, less than: `>` or `<`
* Greater than or equal to `>=`
* Less than or equal to `<=`

Let's try this out. 
We can select all rows that have 80 votes.

In [ ]:
#select all rows that have 80 votes


In [ ]:
# Or we can select all rows that do not contain 80 votes.


In [ ]:
# We can define sets of criteria too:


Next, let's identify all locations in the survey data that have
null (missing or NaN) data values. We can use the `isnull` method to do this.
Each cell with a null value will be assigned a value of  `True` in the new
boolean object.

In [ ]:
pd.isnull(gal_df)

To select the rows where there are null values,  we can use 
the mask as an index to subset our data as follows:

```python
#To select just the rows with NaN values, we can use the .any method
gal_df[pd.isnull(gal_df).any(axis=1)]
```

We can run isnull on a particular column too. What does the code below do?

```python
# what does this do?
empty_pE = gal_df[pd.isnull(gal_df['p_e'])]
```

Let's take a minute to look at the statement above. 

We are using the Boolean object as an index. 
We are asking python to select rows that have a `NaN` value for the probability of a galaxy being Elliptical.

In [ ]:
#To select just the rows with NaN values, we can use the .any method
gal_df[pd.isnull(gal_df).any(axis=1)]

In [ ]:
# what does this do?
empty_pE = gal_df[pd.isnull(gal_df['p_e'])]
empty_pE.describe()

### Your turn


1. You can use the `isin` command in python to query a DataFrame based upon a list of values as follows:
   `gal_df[gal_df['class'].isin([listGoesHere])]`. Use the `isin` function to find all Elliptical galaxies of class E0, E1 and E7. How many records contain these values?
2. The `~` symbol in Python can be used to return the OPPOSITE of the selection that you specify in python. It is equivalent to **is not in**. Write a query that selects all Spiral galaxies that are not classed as 'OTHER' an 'EDGE'.
3. Create a new DataFrame that only contains valid (i.e. no NAN probability) observations of Spiral galaxies with over 100 votes.

In [ ]:
# 1 -> count of E0,E1,E7


In [ ]:
# 2 -> all spirals that are not OTHER or EDGE


In [ ]:
# 3 -> spiral probaility is column 'p_s'


# Cleaning our data for machine learning

[[ go back to the top ]](#Table-of-Contents)

## Problem definition

We have been tasked to develop a machine learning model to classify galaxies by their morphology (appearance) from a dataset containing measurements such as:

- redshift
- magnitude in _ugriz_ bands
- exponential scale radius and ellipticity
- de Vaucouleurs scale radius and ellipticity
- stellar mass
- ...
- etc.

Galaxies in the dataset have labels taken from [Galaxy Zoo](https://www.galaxyzoo.org/) DR1 - Table 2. Galaxy Zoo is described in Lintott et al. 2008, MNRAS, 389, 1179 and the data release is described in Lintott et al. 2011, 410, 166. 

We use the final debiased labels to categorise a galaxy as:

- spiral
- elliptical

![Galaxies](http://4.bp.blogspot.com/_rfhv4lPQhSY/TUQjfMLtrxI/AAAAAAAABAo/W3OF0mXZNZc/s1600/spiral%2Bor%2Belliptical%2Bzoo%2B1.jpg)

<div style="text-align:center;font-weight:bold">Figure: A spiral galaxy (left) and elliptical galaxy (right)</div>

Our goal is to train a model that can accurately classify galaxies. We want our model to generalise well. i.e. it can correctly classify unseen galaxies (i.e. galaxies not in our training dataset).

### Dataset

For our introduction to machine learning, we will be using a subset of the data we worked with so far. To be able to try several machine learning algorithms we will be using a data set that combines [Galaxy Zoo DR1](https://www.galaxyzoo.org/) and the Sloan Digital Sky Survey (SDSS) ([using the DR9 SQL search](http://skyserver.sdss.org/dr9/en/tools/search/sql.asp)). 

The data dictionary for this dataset is presented in Table 1. This dataset is limited to the first 5,000 Galaxy Zoo classified galaxies which have spectra in the SDSS database. The debiased fraction of the votes in elliptical and spiral categories is given, along with columns identifying systems classified as spiral, elliptical or uncertain.

<p style="text-align:center;font-weight:bold">Table 1: Data dictionary</p>

| Column           | Description                                                            |
|:-----------------|:-----------------------------------------------------------------------|
| id               | unique SDSS ID composed of [skyVersion, rerun, run, camcol, field, obj]|
| ra               | right ascension  (HMS)                                                 |
| dec              | declination (DMS)                                                      |
| redshift         | redshift                                                               |
| mag_u            | magnitude _u_ band                                                     |
| mag_g            | magnitude _g_ band                                                     |
| mag_r            | magnitude _r_ band                                                     |
| mag_i            | magnitude _i_ band                                                     |
| mag_z            | magnitude _z_ band                                                     |
| deVRad_r         | de Vaucouleurs scale radius fit in _r_ band                            |
| deVAB_r          | ellipticity from de Vaucouleurs fit in _r_ band                        |
| expRad_r         | exponential scale radius fit in _r_ band                               |
| expAB_r          | ellipticity from exponential fit in _r_ band                           |
| stellar_mass     | log galaxy mass (in units of solar mass)                               |
| votes            | number of Galaxy Zoo annotators                                        |
| p_el_debiased    | debiased labelling probability the galaxy is elliptical                |
| p_cs_debiased    | debiased labelling probability the galaxy is spiral                    |
| spiral           | label = spiral galaxy {0=False, 1=True}                                |
| elliptical       | label = elliptical galaxy {0=False, 1=True}                            |
| uncertain        | label = uncertain {0=False, 1=True}                                    |


**Understand and clean the data**

The next step is to explore and familiarise ourselves with the data using the methods we learnedabove. 
Datasets can have errors, and it's important to identify and resolve these errors before rushing in to develop our ML models. Errors in the dataset that aren't resolved will propogate through the machine learning pipeline and will bias our results.

Generally, we're looking to answer the following questions:

* Is there anything wrong with the data?
* Are there any quirks with the data?
* Do I need to clean or remove some of the data?

Let's start by reading the data into a `pandas` DataFrame and inspecting the first few rows.

In [ ]:
# Import pandas and check directory, 
# change directory if you need to
import pandas as pd
import os
os.getcwd()

In [ ]:
# Read the data file, make sure you are trying to access the correct folder/file
data = pd.read_csv('data/galaxies.csv')

# Display the first few rows using the .head() method
data.head()

**Missing Data Values - NaN**

Dealing with missing data values is always a challenge, think about:
* Why values are missing - was it because of a data entry error? Or are they data that someone was unable to collect? 
* How are the missing values represented? Are they empty cells in a spreadsheet, NaN or NA, or otherwise defined?
* How should you treat the NaN? Should the value be 0 or the mean of all measurements, should they be ignored? 

Note: empty cells in a spreadhseet are automatically replaced with NaN by pandas when the data is read into a DataFrame.

 So, one of the first things we should do is to look for is missing data. We can use `pandas` to  display summary statistics for each (numerical) column as well as inspect rows with missing data.

In [ ]:
# The .describe() method returns summary statistics for each (numerical) column
data.describe()

In [ ]:
# Get records containing any missing values
data[data.isnull().any(axis=1)]

Here we see a number of rows that contain missing values (`NaN`). There are many [methods for handling and imputing missing data](http://machinelearningmastery.com/handle-missing-data-python/). However, from our results we see that the galaxies with missing values have been labelled as `uncertain` so we will simply remove these records. 

In [ ]:
# Drop the rows containing any missing values


Let's double check to make sure the records have been removed.

In [ ]:
# Should return 0 rows
data[data.isnull().any(axis=1)]

As for the data with -9999 values, from our domain knowledge we know that this is a flag to signify NaN. So we can treat these as missing values / errors and remove them from our dataset as well.
Also, having galaxies with masses or magnitudes less than 0 does not make sense and we should remove these records, too.

Now, let's look at and remove our data with values <0. 

Be careful, some data columns can be smaller than zero and still be valid measurements.

Make sure to check any metadata given with your data to figure out what is a bad measurements.

In [ ]:
# Get records with ≤ 0 values using iloc
# remember first number in slice is inclusive, second number is exclusive
# Exclude the first  4 columns (id, ra, dec, redshift) and last 5 columns (galaxy labels and probabilities) 


In [ ]:
# Only retain records with > 0 values in our dataset

# Let's check the summary stats again
data.describe()

It is important to construct a good training dataset for developing a robust model. In machine learning, we also refer to the training dataset as the ground truth. Any uncertainities or errors in the ground truth can confuse and reduce the performance of our models.

In order to build a good training dataset, we only want to keep records where we are confident there is agreement between the Galaxy Zoo annotators in labelling galaxies. From the dataset, it appears we can achieved this by:

- Keeping records where the uncertain label column is equal to False (0)

Let's try this method and perform some sanity checking to see if the resulting data subset has debiased probabilities for ellipital and spiral labels ≥ 0.5.

In [ ]:
# 1. Filter records using the uncertain column 



# 2. Filter records using the debiased probabilities ≥ 0.5



# Print the lengths for comparison
print('# records after removing uncertain rows: %d' % (len(f1)))
print('# records with ≥ 0.5 debiased probability: %d' % (len(f2)))

There appears to be a discrepency of 5 records between the two data subsets. Let's investigate further by inspecting these records.

In [ ]:
# Display filter #1 rows that don't appear in the filter #2 dataset


These records have been labelled as spiral galaxies but the debiased probabilities (confidence) for these labels are below the 0.5 confidence threshold. i.e. `p_cs_debiased < 0.5`. Additionally, for the last 2 records, we see that debiased probabilities for labelling the galaxy as elliptical is actually larger than the spiral probabilities which suggests these might be due to labelling errors. 

It appears we can't rely on filtering the dataset on the `uncertain` column alone so we will apply both filters to clean the dataset. Sanity checking your dataset is a step that is regularly overlooked but dedicating time to explore and resolve these errors can (sometimes dramatically) improve the performance of our machine learning models.

We will make a copy of the filtered dataset in the variable `df` (abbreviation for DataFrame) for our clean dataset.

In [ ]:
# use .copy to create and independent copy of the dataframe
df = f2.copy()

Let's clean and prepare our dataset by:

- creating a class column to represent the galaxy type label `{0=elliptical, 1=spiral}`
- removing unwanted columns that won't be used in our analysis

In [ ]:
# Create the class column - do this by renaming the 'spiral' column to 'class'

# Remove unwanted columns
unwanted_columns = 

# Inspect the first few rows of our prepared dataset
df.head()

Next, it's always a good idea to look to have a closer look at our data — especially for outliers. Let's start by printing out some summary statistics about the data set.

In [ ]:
# Display dataset summary statistics
df.describe()

The summary table provides some useful information:

- there are 1,797 galaxies in our dataset
- the mean of the `class` column indicates that ~75.3% of the dataset are spiral galaxies
- min values for the radius columns range from 0.31 to 0.38
- min value for the `stellar_mass` = 8.096
- min value for the `mag` ranging from 15.59 to 18.5

Our cleaned dataset contains 1,797 galaxies and 75.3% of the records are spiral galaxies.

Tables like this are useful when we know that our data should fall in a particular range. e.g. It did not make sense to have zero or negative values for some of our columns. However, it is usually better to visualize the data in some way. Visualization makes outliers and errors immediately stand out, whereas they might go unnoticed in a large table of numbers.


In [ ]:
# Command to show plots inside of the notebook
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sb

Let's create a **scatterplot matrix**. Scatterplot matrices plot the distribution of each column along the diagonal and a scatterplot matrix for the combination of each variable. They make for an efficient tool to look for errors in our data.

Since we have quite a few columns, let's generate two scatterplots for data containing:

1.  magnitudes
2. measurements

In [ ]:
# Magnitudes scatterplot
magnitudes = ['mag_u', 'mag_g', 'mag_r', 'mag_i', 'mag_z']
selected_columns = magnitudes + ['class']
sb.pairplot(df[selected_columns], hue='class', plot_kws={'alpha': 0.3})

From the magnitude scatterplot matrix, we see a potential outlier: the `mag_u` value of an `elliptical` galaxy falls outside its normal range (a value of ~24) and there seems to be an outlier for the spirals as well with values >18 for all(?) magnitudes.

In [ ]:
# Inspect possible outliers


There is nothing obvious that leads us to believe the records are erroneous. Let's keep them in our dataset.

Next we have a look at the distribution of the other measurements in our dataset:

In [ ]:
# Measurements scatterplot
measurements = ['redshift', 'stellar_mass', 'deVRad_r', 'deVAB_r', 'expRad_r', 'expAB_r']
selected_columns = measurements + ['class']
sb.pairplot(df[selected_columns], hue='class', plot_kws={'alpha': 0.3})

From the scatterplot matrix, we see potential outliers: 

- the `stellar_mass` value of a `spiral` galaxy falls outside its normal range (a value of ~8).
- two spiral galaxies with `expRad_r` ≥ 17

Let's inspect the outlier records to determine whether if we should keep them in our training dataset.

In [ ]:
# Inspect possible outliers


There is nothing obvious that leads us to believe the records are erroneous. Let's keep them in our dataset.

Fixing outliers can be difficult. It's often unclear whether the outlier was caused by measurement error, recording the data in improper units, or if the outlier is a real anomaly. For that reason, we should be careful when working with outliers. If we decide to exclude any data, we should document what data was excluded and provide reasons for excluding that data. i.e. The data didn't fit my hypothesis will not stand peer review.


### Writing Out Data to CSV

After all this hard work, we don't want to repeat the data cleaning process every time we work with this dataset. Let's save the cleaned dataset as a separate file and work directly with that data file from now on.

We can use the `to_csv` command to do export a DataFrame in CSV format. 
Note that by default the data will be saved into the current working directory. 
We can save it to a different folder by adding the foldername and a slash before the file name,
either using relative or absolute path names.

```python
# Write DataFrame to CSV 
df.to_csv('galaxies-clean.csv')
```

Check out your working directory to make sure the CSV wrote out properly, and
that you can open it! If you want, try to bring it back into python to make sure
it imports properly.

```python	
# for kicks read our output back into python and make sure all looks good
new_output = pd.read_csv('galaxies-clean.csv', keep_default_na=False, na_values=["NA"])
```

In [ ]:
# Save the clean dataset to a file (try saving it into the correct folder for tomorrows/DAY3 lesson)
df.to_csv('galaxies-clean.csv', index=False)

# Exploratory analysis
[[ go back to the top ]](#Table-of-Contents)

Exploratory analysis is the step where we start delving deeper into the data set beyond the outliers and errors. We'll be looking to answer questions such as:

- How is my data distributed?
- Are there any correlations in my data?
- Are there any confounding factors that explain these correlations?

This is the stage where we plot the data in as many ways as possible. Create many charts, but don't bother making them pretty as these charts are for internal use.

For completeness, let's generate a scatterplot for all features in one figure.

In [ ]:
# Command to show plots inside of the notebook
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sb

magnitudes = ['mag_u', 'mag_g', 'mag_r', 'mag_i', 'mag_z']
measurements = ['redshift', 'stellar_mass', 'deVRad_r', 'deVAB_r', 'expRad_r', 'expAB_r']


# Plot scatterplot matrix with all features
selected_columns = measurements + magnitudes + ['class']
sb.pairplot(df[selected_columns], hue='class', plot_kws={'alpha': 0.3})

We can make a number of observations from this plot:

1. `reshift`, `stellar_mass`, `mag_u` and `mag_g` follow a normal distribution. This is something to keep in mind if we use modelling methods that assume the data is normally distributed.

2. There is a strong positive correlation between the `expAB_r` and `deVAB_r` which indicates we might not need to use both of these features in our model.

3. There is a strong positive correlation between the `mag_r`, `mag_i` and `mag_z`.

4. There is a positive correlation between `mag_u` and `mag_g` with the other magnitude bands but not as strong as in observation #3. This suggests these features might provide added information not present in the `mag_r`, `mag_i` and `mag_z` bands.

5. There are no clear pair-wise combinations of features that can easily seperate the two classes. 

Distinguishing between `spiral` and `elliptical` galaxies might be difficult given how much these features interrelate.

We can also make **violin plots** of the data to compare the measurement distributions of the classes. Violin plots contain the same information as [box plots](https://en.wikipedia.org/wiki/Box_plot), but also scales the box according to the density of the data.

In [ ]:
# General violin plots of all features
selected_columns = measurements + magnitudes + ['class']
plt.figure(figsize=(13, 13))
for column_index, column in enumerate(selected_columns):
    if column == 'class':
        continue
    plt.subplot(3, 4, column_index + 1)
    sb.violinplot(x='class', y=column, data=df[selected_columns])

There are numerous errors and possible scenarios you may face while exploring and cleaning your data.

The general takeaways here should be:

- Handle missing data: replace it if you can or drop it
- Ensure your data is encoded properly
- Check if your data falls within the expected range and use domain knowledge whenever possible to define that expected range
- Avoid tidying your data manually because that is not easily reproducible
- Use code as a record of how you tidied your data
- Plot everything you can about the data at this stage of the analysis so you can visually confirm everything looks correct

# Supplement: Merging DataFrames

[[ go back to the top ]](#Table-of-Contents)

In many "real world" situations, the data that we want to use come in multiple
files. We often need to combine these files into a single DataFrame to analyze
the data. The pandas package provides [various methods for combining
DataFrames](http://pandas.pydata.org/pandas-docs/stable/merging.html) including
`merge` and `concat`.

To work through the examples below, we first need to load the galaxy zoo and
SDSS files into pandas DataFrames. In iPython:

``` python
import pandas as pd
gal_df = pd.read_csv('GalaxyZooSub.csv',
                         keep_default_na=False, na_values=["NA"])
sdss = pd.read_csv('SDSS_blue_centre_query.csv',
                         keep_default_na=False, na_values=[""])
gal_df.dtypes
sdss.dtypes
```

In [ ]:
import pandas as pd
gal_df = pd.read_csv('data/GalaxyZooSub.csv',
                         keep_default_na=False, na_values=["NA"])
sdss = pd.read_csv('data/SDSS_blue_centre_query.csv',
                         keep_default_na=False, na_values=[""])


In [ ]:
gal_df.dtypes

In [ ]:
sdss.dtypes

Take note that the `read_csv` method we used can take some additional options which
we didn't use previously. Many functions in python have a set of options that
can be set by the user if needed. In this case, we have told Pandas to assign
empty values in our CSV to NaN `keep_default_na=False, na_values=[""]`.
[http://pandas.pydata.org/pandas-docs/dev/generated/pandas.io.parsers.read_csv.html](More
about all of the read_csv options here.)


## Concatinating

We can use the `concat` function in Pandas to append either columns or rows from
one DataFrame to another.  Let's grab two subsets of our data to see how this
works.


In [ ]:
# read in first 10 lines of gal_df table
gal_sub=gal_df.head(10)
gal_sub

In [ ]:
# grab the last 10 rows (minus the last one)
gal_last10=gal_df[-11:-1]
gal_last10

In [ ]:
#reset the index values to the second dataframe appends properly
# drop=True option avoids adding new index column with old index values
gal_last10 = gal_last10.reset_index(drop=True)
gal_last10

When we concatenate DataFrames, we need to specify the axis. `axis=0` tells
Pandas to stack the second DataFrame under the first one. It will automatically
detect whether the column names are the same and will stack accordingly.
`axis=1` will stack the columns in the second DataFrame to the RIGHT of the
first DataFrame. To stack the data vertically, we need to make sure we have the
same columns and associated column format in both datasets. When we stack
horizonally, we want to make sure what we are doing makes sense (ie the data are
related in some way).


In [ ]:
# stack the DataFrames on top of each other
vertical_stack = pd.concat([gal_sub,gal_last10], axis=0)
vertical_stack

In [ ]:
# place the DataFrames side by side
horizontal_stack = pd.concat([gal_sub,gal_last10], axis=1)
horizontal_stack

### Your turn 
**Row Index Values and Concat**

Have a look at the `vertical_stack` dataframe? Notice anything unusual?
The row indexes for the two data frames `survey_sub` and `survey_sub_last10`
have been repeated. We can reindex the new dataframe using the `reset_index()` method.

# Joining DataFrames

When we concatenated our DataFrames we simply added them to each other -
stacking them either vertically or side by side. Another way to combine
DataFrames is to use columns in each dataset that contain common values (a
common unique id). Combining DataFrames using a common field is called
"joining". The columns containing the common values are called "join key(s)".
Joining DataFrames in this way is often useful when one DataFrame is a "lookup
table" containing additional data that we want to include in the other. 

For example, the `SDSS_blue_centre_query.csv` file that we've been loaded is a query from the SDSS DR7 database for 1000 galaxies with blue centres (things we would expect to be Spirals). 
This table contains the ID, postition, redshift and r-band magnitude as well as size measurements.


## Identifying join keys

To identify appropriate join keys we first need to know which field(s) are
shared between the files (DataFrames). We might inspect both DataFrames to
identify these columns. If we are lucky, both DataFrames will have columns with
the same name that also contain the same data. If we are less lucky, we need to
identify a (differently-named) column in each DataFrame that contains the same
information.

Check the column names for gal_df and sdss

In [ ]:
print(gal_df.columns.values)
print(sdss.columns.values)


In our example, the join key is the column containing the object
identifier. We could also use 'ra' and 'dec', however, they are not presented in the same units thus joining on id will be esier.

There are [different types of joins](http://blog.codinghorror.com/a-visual-explanation-of-sql-joins/), so we
also need to decide which type of join makes sense for our analysis.

## Inner joins

The most common type of join is called an _inner join_. An inner join combines
two DataFrames based on a join key and returns a new DataFrame that contains
**only** those rows that have matching values in *both* of the original
DataFrames. 

Inner joins yield a DataFrame that contains only rows where the value being
joins exists in BOTH tables. An example of an inner join, adapted from [this
page](http://blog.codinghorror.com/a-visual-explanation-of-sql-joins/) is below:

![Inner join -- courtesy of codinghorror.com](http://blog.codinghorror.com/content/images/uploads/2007/10/6a0120a85dcdae970b012877702708970c-pi.png)

The pandas function for performing joins is called `merge` and an inner join is
the default option:  

In [ ]:
merged_inner = pd.merge(left=gal_df,right=sdss, left_on='id', right_on='objID')
# if both column names were 'id', we could skip the `left_on`
# and `right_on` arguments and still get the same result

In [ ]:
# what's the size of the output data?
merged_inner.shape

In [ ]:
merged_inner

The result of an inner join of `gal_df` and `sdss` is a new DataFrame
that contains the combined set of columns from `gal_df` and `sdss`. It
*only* contains rows that have an ID that is the same in
both the `gal_df` and `sdss` DataFrames. 

In other words, if a row in
`gal_df` has a value of `id` that does *not* appear in the `objID`
column of `sdss`, it will not be included in the DataFrame returned by an
inner join.  Similarly, if a row in `sdss` has a value of `objID`
that does *not* appear in the `id` column of `gal_df`, that row will not
be included in the DataFrame returned by an inner join.

The two DataFrames that we want to join are passed to the `merge` function using
the `left` and `right` argument. The `left_on='id'` argument tells `merge`
to use the `id` column as the join key from `gal_df` (the `left`
DataFrame). Similarly , the `right_on='objID'` argument tells `merge` to
use the `objID` column as the join key from `sdss` (the `right`
DataFrame). For inner joins, the order of the `left` and `right` arguments does
not matter.

The result `merged_inner` DataFrame contains all of the columns from `survey_sub` as well as all the columns from `sdss`.

Notice that `merged_inner` has way fewer rows than `gal_df`. This is an
indication that there were rows in `gal_df` with value(s) for `id` that
do not exist as value(s) for `objID` in `sdss`.
 
## Left joins

What if we want to add information from `sdss` to `gal_df` without
losing any of the information from `gal_df`? In this case, we use a different
type of join called a "left outer join", or a "left join".

Like an inner join, a left join uses join keys to combine two DataFrames. Unlike
an inner join, a left join will return *all* of the rows from the `left`
DataFrame, even those rows whose join key(s) do not have values in the `right`
DataFrame.  Rows in the `left` DataFrame that are missing values for the join
key(s) in the `right` DataFrame will simply have null (i.e., NaN or None) values
for those columns in the resulting joined DataFrame.

Note: a left join will still discard rows from the `right` DataFrame that do not
have values for the join key(s) in the `left` DataFrame.

![Left Join](http://blog.codinghorror.com/content/images/uploads/2007/10/6a0120a85dcdae970b01287770273e970c-pi.png)

A left join is performed in pandas by calling the same `merge` function used for
inner join, but using the `how='left'` argument:

In [ ]:
merged_left = pd.merge(left=gal_df,right=sdss, left_on='id', right_on='objID', how='left')

merged_left

The result DataFrame from a left join (`merged_left`) looks very much like the
result DataFrame from an inner join (`merged_inner`) in terms of the columns it
contains. However, unlike `merged_inner`, `merged_left` contains the **same
number of rows** as the original `gal_df` DataFrame. When we inspect
`merged_left`, we find there are rows where the information that should have
come from `sdss` (e.g., `z`, `r`, and `expRad_r`) is
missing (they contain NaN values):

In [ ]:
len(merged_left[ pd.isnull(merged_left.z) ])


These rows are the ones where the value of `id` from `sdss` does not occur in `gal_df`.


## Other join types

The pandas `merge` function supports two other join types:

* Right (outer) join: Invoked by passing `how='right'` as an argument. Similar
  to a left join, except *all* rows from the `right` DataFrame are kept, while
  rows from the `left` DataFrame without matching join key(s) values are
  discarded.
* Full (outer) join: Invoked by passing `how='outer'` as an argument. This join
  type returns the all pairwise combinations of rows from both DataFrames; i.e.,
  the result DataFrame will `NaN` where data is missing in one of the dataframes. This join type is
  very rarely used.
